In [55]:
import pandas as pd
import pickle
import os


ROOT_FOLDER = os.path.dirname(os.getcwd())
FEATHER_PATH = os.path.join(ROOT_FOLDER, "data", "processed", "merged_df_cleaned.feather") #I created this file from the Random-Forest model. It is the cleaned and processed dataset
PICKLE_PATH = os.path.join(ROOT_FOLDER, "models", "Readmission_Model.pkl")

# Important features used by the model
important_features = ['icu_los', 'Glucose_max', 'Hematocrit_max', 'Respiratory Rate_mean',
       'Creatinine_max', 'Sodium_min', 'urine_last_1d',
       'Platelet Count_median', 'Glucose_median', 'Strength L Leg_max',
       'Magnesium_median', 'Magnesium_max', 'MCHC_std', 'Potassium_min',
       'Glucose_min', 'Anion Gap_max', 'Respiratory Rate_std',
       'White Blood Cells_max', 'Phosphorous_median',
       'Non Invasive Blood Pressure mean_max',
       'Alanine Aminotransferase (ALT)_median']

# Function to load the model
def load_model(pickle_path):
    with open(pickle_path, 'rb') as file:
        model = pickle.load(file)
    return model

# Function to prepare input data for a given stay_id
def prepare_input_data(stay_id, df):
  
    data = df[df['icu_stay_id'] == stay_id].reset_index(drop=True)
    
    return data[important_features]

# Function to predict readmission
def predict_readmission(stay_id, model, df):
    if stay_id not in df['icu_stay_id'].values:
        return None, None  
    
    input_data = prepare_input_data(stay_id, df)
    
    if input_data.empty:
        return None, None 
    
   
    prediction_proba = model.predict_proba(input_data)[:, 1]  
    prediction_percentage = prediction_proba * 100  
    return prediction_proba, prediction_percentage


def get_prediction_message(stay_id, model, df):
    readmission_probability, readmission_percentage = predict_readmission(stay_id, model, df)
    
    if readmission_probability is None:
        message = f"No data found or stay_id {stay_id} not found."
        success = False
    else:
        message = (f"Predicted readmission probability for stay_id {stay_id}: "
                   f"{readmission_probability[0]:.2f}\n"
                   f"Predicted readmission percentage for stay_id {stay_id}: "
                   f"{readmission_percentage[0]:.2f}%")
        success = True
    
    return message, success

if __name__ == "__main__":
    
    merged_df_cleaned = pd.read_feather(FEATHER_PATH)
    
    model = load_model(PICKLE_PATH)
    
    stay_id = input("Enter stay_id: ")
    
    message, success = get_prediction_message(stay_id, model, merged_df_cleaned)
    
    print(message)


Predicted readmission probability for stay_id 37067082: 0.78
Predicted readmission percentage for stay_id 37067082: 78.00%
